In [1]:
import re
import qiskit
import numpy as np
from typing import List
from qiskit import *


In [2]:
from circuit_generator import construct_qcc_circuit, simulate_expectationval, update_observables \
             ,fc_tree_circuit, fc_tree_commute_circuit, fc_tree_commute_lookahead_circuit
from utils import compare_lists


In [3]:
# test_observable = 'ZZZZZZ'
# test_paulis = ['XXXXXY', 'XXXIYI', 'IXIXXY', 'IXIIYI', 'IXXIXY', 'XXIXYI', 'IIIXIY', 'XIYIII']
# test_params = [0.0944527, 0.04799566, -0.0590973, -0.05908328, 0.04114604, 0.02695483, 0.02604318, 0.03485649]

In [4]:
# test_observable = 'ZZZZZZZZ'
# test_paulis = ['XXIIIIXY', 'IIXXXYII', 'IXXIXIIY', 'XIIXIXYI',
#                   'XXIIXYII', 'IXIXIXIY', 'XIXIXIYI', 'IIXXIIXY']
# test_params = [1.16692654, 0.27223177, -0.93402707, -0.92067998, 0.06852241, -0.42444632, -0.41270851, -0.01068001]

In [5]:
# #simplified two strings:
# test_observables = ['XXXXXX', 'YYYYYY', 'XYXYXY', 'YXYXYX', 'YYYXXX', 'XXXYYY', 'ZZZZZZ', 'ZZIIII', 'IIZZII', 'IIIIZZ', 'XXXXXZ','XXXXZZ', 'XXXZZZ','XXZZZZ','XZZZZZ','ZZZXXX']
# test_paulis = ['XXXXXY', 'XXXIYI', 'IXIXXY', 'IXIIYI', 'IXXIXY', 'XXIXYI', 'IIIXIY', 'XIYIII']
# test_params = [0.0944527, 0.04799566, -0.0590973, -0.05908328, 0.04114604, 0.02695483, 0.02604318, 0.03485649]

In [6]:
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers

In [7]:
test_observables = ['XXXXXXXXXXXX', 'ZZZZZZZZZZZZ']
test_paulis = generate_UCCSD_entanglers(6,12)[0:40]
test_params = [0.01 * i for i in range(len(test_paulis))]

First run the original circuit and simulate

In [8]:
origin_qc = construct_qcc_circuit(entanglers = test_paulis, params = test_params, barrier=False)
origin_qc.count_ops()['cx']

560

In [9]:
origin_qiskit = transpile(origin_qc, optimization_level = 3, basis_gates = ["cx", "sx", "x", "rz"])
origin_qiskit.count_ops()['cx']

270

In [10]:
orign_expect_vals = []
for obs in test_observables:
    expectation_val = simulate_expectationval(origin_qc, observable=obs, shots=100000)
    orign_expect_vals.append(expectation_val)

XXXXXXXXXXXX
ZZZZZZZZZZZZ


In [11]:
orign_expect_vals

[-0.0009, 1.0]

Now let's generate the optimized fully connected circuits, 
there are four versions: 1. generate_fc_tree_circuit: Just local search
2. generate_fc_tree_commute_circuit: local search + commute
3. generate_fc_tree_lookahead_circuit: local search + lookahead
4. generate_fc_tree_lookahead_commute_circuit: local search + commute + lookahead

In [12]:
opt_qc, append_clifford, opt_paulis, opt_params = fc_tree_circuit(entanglers = test_paulis, params = test_params, barrier=False)

In [13]:
opt_qc.count_ops()['cx']

120

In [14]:
opt_qiskit = transpile(opt_qc, optimization_level = 3, basis_gates = ["cx", "sx", "x", "rz"])
opt_qiskit.count_ops()['cx']

119

In [15]:
updated_signs, updated_observables = update_observables(test_observables, [append_clifford])
print(updated_observables)

['ZXZIZZXXZXZZ', 'IXIZZYIZIXXX']


In [16]:
opt_expect_vals = []
for idx, obs in enumerate(updated_observables):
    expectation_val = simulate_expectationval(opt_qc, observable=obs, shots=100000)
    if updated_signs[idx] == '+1':
        updated_sign = 1
    elif updated_signs[idx] == '-1':
        updated_sign = -1
    else:
        raise Exception("incorrect sign")
    opt_expect_vals.append(updated_sign * expectation_val)

ZZXZXXZZZZXZ
XXXZZZYZZZXZ


In [17]:
opt_expect_vals

[0.00056, 1.0]

Run the optimized versoin with lookahead and commute

In [18]:
opt_qc2, append_clifford2, sorted_entanglers2 = fc_tree_commute_lookahead_circuit(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size = 5)
opt_qc2.count_ops()['cx']

58

In [19]:
opt_qiskit2 = transpile(opt_qc2, optimization_level = 3, basis_gates = ["cx", "sx", "x", "rz"])
opt_qiskit2.count_ops()['cx']

53

In [20]:
updated_signs2, updated_observables2 = update_observables(test_observables, [append_clifford2])
print(updated_observables2)

['ZZYXZXYZIYYY', 'IIIYIIIIXXII']


In [21]:
opt_expect_vals2 = []
for idx, obs in enumerate(updated_observables2):
    expectation_val = simulate_expectationval(opt_qc2, observable=obs, shots=100000)
    if updated_signs2[idx] == '+1':
        updated_sign = 1
    elif updated_signs2[idx] == '-1':
        updated_sign = -1
    else:
        raise Exception("incorrect sign")
    opt_expect_vals2.append(updated_sign * expectation_val)

YYYZZYXZXYZZ
ZZXXZZZZYZZZ


In [22]:
opt_expect_vals2

[0.00512, 1.0]

Validate the results

In [27]:
try:
    compare_lists(orign_expect_vals, opt_expect_vals, tolerance = 0.01)
    print("Lists are within the tolerance.")
except ValueError as e:
    print(e)

Lists are within the tolerance.


In [28]:
try:
    compare_lists(orign_expect_vals, opt_expect_vals2, tolerance = 0.01)
    print("Lists are within the tolerance.")
except ValueError as e:
    print(e)

Lists are within the tolerance.
